In [5]:
import pandas as pd
import pickle
import numpy as np
dataset = pd.read_csv("../datasets/merged_dataset.csv", parse_dates=["founded_at"], dayfirst=True)

columns = ["id", "object_id", "funding_round_type", "raised_amount_usd", "entity_type", "name", "status", "founded_at",
            "region", "invested_companies", "funding_total_usd", "country_code", "category_code", "tag_list"]
# df.info(show_counts=True)
df = dataset.copy()
df=df[columns]
# df.info(show_counts=True)

# df.to_csv("../datasets/startup_investments.csv", index=False)



In [7]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

y = df.funding_total_usd

features = ["region",  "funding_round_type", "category_code"]

X = df[features]

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
# ohe.fit_transform(X[["region"]])

ore = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, categories=[["series-a", "series-b", "series-c+", "angel", "venture", "private-equity", "post-ipo", "crowdfunding", "other"]])
# ore.fit_transform(X[["funding_round_type"]])
# X["funding_round_type"].unique()

column_transform = make_column_transformer(
  (ohe, ["region", "category_code"]),
  (ore, ["funding_round_type"])
)

column_transform.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# print("X_train shape", X_train.shape)
# print("Y_train shape", Y_train.shape)
# print("X_test shape", X_test.shape)
# print("Y_test shape", Y_test.shape)
  
print(X_test)

              region funding_round_type category_code
27942         SF Bay           series-a      hardware
15618      Milwaukee              angel       finance
41604        Roncade              angel           web
29708         Mahwah            venture   advertising
40249         Moscow              angel        mobile
...              ...                ...           ...
29842        Toronto              angel     education
48707         SF Bay           series-a     education
12829    Los Angeles          series-c+       biotech
8482          Dublin              angel      software
26377  Washington DC              other      software

[15788 rows x 3 columns]


In [8]:
dtm = DecisionTreeRegressor()
dtm_pipeline = make_pipeline(column_transform, dtm)

dtm_pipeline.fit(X_train, Y_train)

filename = 'model.sav'
pickle.dump(dtm_pipeline, open(filename, 'wb'))

dtm_pipeline.score(X_test, Y_test)

# print(X_test)
# predictions = dtm_pipeline.predict(X_test)
# print("First 5 DTM predictions", list(predictions[:5]))


0.2787432192455682

In [ ]:
dtm_pipeline.fit(X, y)
pickle.dump(dtm_pipeline, open("../backend/models/trained_model.sav", "wb"))


In [ ]:
# load the model from disk
loaded_model = pickle.load(open("../backend/models/trained_model.sav", 'rb'))

user_input_1 = ["SF Bay", "operating", "web"]
user_input_2 = ["series a"]

input = np.concatenate((user_input_1, user_input_2), axis=0).reshape(1, -1)
input = pd.DataFrame(input, columns = ["region", "status", "category_code", "funding_round_type"])


prediction = loaded_model.predict(input)[0]
prediction


15186544.425531914